<a href="https://colab.research.google.com/github/oliveralbrecht94/ATDIT2/blob/master/Solution/Task%20E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [420]:
import tensorflow as tf
import numpy as np
import skimage as ski
import matplotlib.pyplot as plt
from scipy import stats
import scipy.spatial.distance

plt.set_cmap(None)
plt.gray()

<Figure size 432x288 with 0 Axes>

In [0]:
cifar_train = tf.keras.datasets.cifar10.load_data()[0]
train_images, train_labels = cifar_train[0], cifar_train[1]

In [0]:
cifar_test = tf.keras.datasets.cifar10.load_data()[1]
test_images, test_labels = cifar_test[0], cifar_test[1]

In [0]:
train_labels_list = []
for i in range(train_labels.shape[0]):
    train_labels_list.append(train_labels[i][0])

In [0]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

In [0]:
y_train = train_labels
y_test = test_labels

# Images into grey

In [0]:
x_train_gray = []

for i in range(train_images.shape[0]):
    x_train_gray.append(rgb2gray(train_images[i]))
    
x_train_gray = np.array(x_train_gray)

In [0]:
x_test_gray = []

for i in range(test_images.shape[0]):
    x_test_gray.append(rgb2gray(test_images[i]))
    
x_test_gray = np.array(x_test_gray)

In [428]:
train_images.shape

(50000, 32, 32, 3)

# Preparing Dataset

In [429]:

n_train = train_images.shape[0]
n_test = test_images.shape[0]
m = train_images.shape[1]**2


# Transform to 1D vector from each matrix with the pixels of an image in order to calculate distances.
gray_image = x_train_gray.reshape([n_train, m])
gray_test_image = x_test_gray.reshape([n_test, m])
gray_image.shape

(50000, 1024)

In [430]:
size_image_sample = 20000
index = np.random.randint(n_train, size = size_image_sample) 
image_sample = gray_image[index]
labels_sample = train_labels[index]
image_sample.shape

(20000, 1024)

In [0]:
size_test_sample = 1000
index = np.random.randint(n_test, size = size_test_sample)
test_image_sample = gray_test_image[index]
test_labels_sample = test_labels[index]

In [0]:
labels_sample = labels_sample.reshape([size_image_sample])
test_labels_sample = test_labels_sample.reshape([size_test_sample])

# KNN - Setting up K and calculate distances

In [0]:
# setting up the k value 
k = 2 
# k = 4
#k = 8 
# calculating the distance between each vector in dependency of k and save it in an arra
distances = scipy.spatial.distance.cdist(image_sample, test_image_sample, metric='euclid')
# calculates the next k points around the vector
index_nearest = np.argpartition(distances, k, axis = 0)[:k] 

In [434]:
# saves the neighbors of the vector in dependency of k
knn_neighbors = labels_sample[index_nearest] 
# show the neighbors
print(knn_neighbors)

[[0 0 6 ... 2 2 9]
 [0 2 6 ... 8 5 1]]


In [435]:
# is looking for the most frequent value in the array/vector
knn_result = stats.mode(knn_neighbors)[0]
knn_result = knn_result[0]
# shows the most frequent value
knn_result

array([0, 0, 6, 4, 1, 5, 8, 0, 2, 0, 8, 0, 0, 0, 4, 5, 2, 0, 2, 7, 0, 0,
       2, 0, 2, 5, 5, 2, 0, 2, 3, 8, 0, 1, 4, 0, 3, 2, 8, 5, 4, 0, 0, 4,
       4, 1, 8, 2, 3, 4, 8, 8, 2, 8, 2, 2, 1, 2, 8, 1, 2, 0, 4, 8, 2, 2,
       0, 8, 0, 0, 4, 8, 2, 8, 2, 2, 3, 0, 3, 8, 0, 0, 0, 0, 4, 2, 0, 3,
       0, 2, 0, 4, 7, 0, 3, 3, 6, 8, 2, 0, 1, 2, 2, 7, 3, 6, 5, 4, 0, 7,
       0, 2, 8, 1, 0, 1, 1, 0, 4, 8, 0, 3, 2, 2, 4, 2, 1, 6, 4, 0, 2, 0,
       3, 0, 2, 8, 7, 6, 0, 6, 0, 0, 8, 8, 8, 0, 2, 3, 4, 0, 8, 8, 2, 0,
       2, 0, 0, 4, 0, 2, 4, 6, 4, 2, 8, 6, 0, 7, 3, 7, 8, 2, 8, 4, 0, 0,
       2, 2, 2, 0, 0, 8, 3, 0, 2, 8, 3, 7, 2, 0, 2, 2, 8, 4, 0, 8, 3, 4,
       0, 0, 4, 5, 3, 0, 0, 2, 7, 4, 2, 2, 2, 0, 2, 0, 1, 9, 4, 3, 3, 2,
       4, 5, 8, 0, 8, 0, 0, 2, 6, 7, 3, 0, 4, 3, 3, 4, 7, 1, 0, 0, 0, 3,
       0, 0, 2, 4, 2, 5, 3, 5, 3, 2, 0, 6, 2, 5, 8, 0, 0, 0, 3, 4, 2, 6,
       2, 4, 4, 2, 2, 5, 0, 1, 5, 4, 4, 4, 5, 4, 5, 4, 1, 2, 2, 2, 2, 2,
       4, 6, 0, 3, 1, 2, 2, 0, 5, 6, 2, 8, 2, 0, 2,

# Calculating error rate

In [0]:
errors = 0
# compares the two arrays and is looking for matches of the values? Right = true / Wrong = false
error_rate_matrix = (test_labels_sample == knn_result)
for b in error_rate_matrix:
    if b == False:
        errors += 1


In [437]:
np.unique(test_labels_sample, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([ 93, 103, 108,  98, 114, 105,  88,  84, 110,  97]))

In [438]:
# Calculation of the error rate
_rate = (errors / size_test_sample) * 100. 
print("Global error rate for k = " +str(k)+" "+str(_rate) + " %")

Global error rate for k = 2 75.9 %


Global error rate for k = 2 86.0 %


Global error rate for k = 4 78.0 %


Global error rate for k = 8 81.0 %

# Get the individual errorrate

In [0]:
indi_error ={0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0}
total = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0}
for i in range(len(test_labels_sample)):
  if test_labels_sample[i]!= knn_result[i]:
    # speicher das Bild falsch war
    indi_error[test_labels_sample[i]] +=1
    total[test_labels_sample[i]] +=1
  else:
    # speicher das Bild richtig war
    total[test_labels_sample[i]] +=1

In [0]:
errorrate = {}
for j in range(0,10):
  errorrate[j] = (indi_error[j]/total[j]) * 100
  #print(errorrate[j])


In [441]:
for i in range(len(errorrate)):
  print("Error rate for "+ str(class_names[i]) +" is "+ str(errorrate[i])+"%")

Error rate for airplane is 49.46236559139785%
Error rate for automobile is 77.66990291262135%
Error rate for bird is 67.5925925925926%
Error rate for cat is 83.6734693877551%
Error rate for deer is 71.9298245614035%
Error rate for dog is 84.76190476190476%
Error rate for frog is 88.63636363636364%
Error rate for horse is 90.47619047619048%
Error rate for ship is 57.27272727272727%
Error rate for truck is 92.78350515463917%


**indiv error rate for k = 2:**

* Error rate for airplane is 53.333333333333336%
* Error rate for automobile is 100.0%
* Error rate for bird is 100.0%
* Error rate for cat is 88.88888888888889%
* Error rate for deer is 92.85714285714286%
* Error rate for dog is 58.333333333333336%
* Error rate for frog is 84.61538461538461%
* Error rate for horse is 100.0%
* Error rate for ship is 91.66666666666666%
* Error rate for truck is 100.0%

**indiv error rate for k = 4**

* Error rate for airplane is 40.0%
* Error rate for automobile is 100.0%
* Error rate for bird is 75.0%
* Error rate for cat is 88.88888888888889%
* Error rate for deer is 92.85714285714286%
* Error rate for dog is 66.66666666666666% 
* Error rate for frog is 76.92307692307693% 
* Error rate for horse is 87.5%
* Error rate for ship is 91.66666666666666%
* Error rate for truck is 100.0%

**indiv error rate for k = 8**

* Error rate for airplane is 53.333333333333336%
* Error rate for automobile is 100.0%
* Error rate for bird is 75.0%
* Error rate for cat is 100.0%
* Error rate for deer is 78.57142857142857%
* Error rate for dog is 50.0%
* Error rate for frog is 61.53846153846154%
* Error rate for horse is 100.0%
* Error rate for ship is 91.66666666666666%
* Error rate for truck is 100.0%

**Does knn work similarly good?**

Knn works at CIFAR extremely inaccurate as you can see from the high error rates. With every test, the error rate is over 75%, while with MNIST, the error rate is a few percent.



# Kmeans - Setting up k and calculating distances

In [0]:
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
clusters = ["Cluster 0", "Cluster 1", "Cluster 2", "Cluster 3", "Cluster 4", "Cluster 5", "Cluster 6", "Cluster 7", "Cluster 8", "Cluster 9"]

In [443]:
k = 10
# init centroids
idx_centroid = np.random.randint(size_image_sample, size=k)
centroids = image_sample[idx_centroid]


idx_centroid

array([ 9147, 19476,  5068, 18406, 18449, 10036,  7264,   111, 19818,
       19189])

In [444]:
# calculate distances
dists = scipy.spatial.distance.cdist(image_sample, centroids, metric='euclid')
dists.shape

(20000, 10)

In [445]:
#find nearest ones
labels = np.argmin(dists, axis=1)
labels

array([8, 8, 8, ..., 4, 2, 7])

In [446]:
np.unique(labels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [0]:
# get new centroids
list = [image_sample[labels == i].mean(0) for i in range(k)]
centroids = np.stack(list)

In [0]:
# do the iteraions of kmean
#iteration = 10
iteration = 100
#iteration = 1000
for j in range(iteration):
    
    # calc distances
    dists = scipy.spatial.distance.cdist(image_sample, centroids, metric = 'euclid')
    
    # label according to nearest centroids
    centroid_labels = np.argmin(dists, axis = 1)
    
    # get new centroids
    centroid_list = [image_sample[centroid_labels == i].mean(0) for i in range(k)]
    centroids = np.stack(centroid_list)

In [0]:
# select all data in cluster_n
cluster_0 = image_sample[labels == 0]
cluster_1 = image_sample[labels == 1]
cluster_2 = image_sample[labels == 2]
cluster_3 = image_sample[labels == 3]
cluster_4 = image_sample[labels == 4]
cluster_5 = image_sample[labels == 5]
cluster_6 = image_sample[labels == 6]
cluster_7 = image_sample[labels == 7]
cluster_8 = image_sample[labels == 8]
cluster_9 = image_sample[labels == 9]



In [0]:
labels_cluster_0 = labels_sample[centroid_labels == 0]
labels_cluster_1 = labels_sample[centroid_labels == 1]
labels_cluster_2 = labels_sample[centroid_labels == 2]
labels_cluster_3 = labels_sample[centroid_labels == 3]
labels_cluster_4 = labels_sample[centroid_labels == 4]
labels_cluster_5 = labels_sample[centroid_labels == 5]
labels_cluster_6 = labels_sample[centroid_labels == 6]
labels_cluster_7 = labels_sample[centroid_labels == 7]
labels_cluster_8 = labels_sample[centroid_labels == 8]
labels_cluster_9 = labels_sample[centroid_labels == 9]

In [455]:
print("Numbers in Cluster 0: "+str(np.unique(labels_cluster_0)))
print("Numbers in Cluster 1: "+str(np.unique(labels_cluster_1)))
print("Numbers in Cluster 2: "+str(np.unique(labels_cluster_2)))
print("Numbers in Cluster 3: "+str(np.unique(labels_cluster_3)))
print("Numbers in Cluster 4: "+str(np.unique(labels_cluster_4)))
print("Numbers in Cluster 5: "+str(np.unique(labels_cluster_5)))
print("Numbers in Cluster 6: "+str(np.unique(labels_cluster_6)))
print("Numbers in Cluster 7: "+str(np.unique(labels_cluster_7)))
print("Numbers in Cluster 8: "+str(np.unique(labels_cluster_8)))
print("Numbers in Cluster 9: "+str(np.unique(labels_cluster_9)))

Numbers in Cluster 0: [0 1 2 3 4 5 6 7 8 9]
Numbers in Cluster 1: [0 1 2 3 4 5 6 7 8 9]
Numbers in Cluster 2: [0 1 2 3 4 5 6 7 8 9]
Numbers in Cluster 3: [0 1 2 3 4 5 6 7 8 9]
Numbers in Cluster 4: [0 1 2 3 4 5 6 7 8 9]
Numbers in Cluster 5: [0 1 2 3 4 5 6 7 8 9]
Numbers in Cluster 6: [0 1 2 3 4 5 6 7 8 9]
Numbers in Cluster 7: [0 1 2 3 4 5 6 7 8 9]
Numbers in Cluster 8: [0 1 2 3 4 5 6 7 8 9]
Numbers in Cluster 9: [0 1 2 3 4 5 6 7 8 9]


In [0]:
def percentagecalculator(labels_cluster, class_names, c):
    cluster = stats.mode(labels_cluster)
    labels, count = np.unique(labels_cluster, return_counts=True)
    highest_idx = 0
    for i in range(len(labels)):
      if count[i] > count[highest_idx]:
        highest_idx = i
    per = float(cluster[1]) / float(labels_cluster.shape[0]) * 100
    print("Cluster " + str(c) + " contains " + str(round(per, 2)) + " %  " + str(labels[highest_idx]))

In [457]:
percentagecalculator(labels_cluster_0, class_names, 0)
percentagecalculator(labels_cluster_1, class_names, 1)
percentagecalculator(labels_cluster_2, class_names, 2)
percentagecalculator(labels_cluster_3, class_names, 3)
percentagecalculator(labels_cluster_4, class_names, 4)
percentagecalculator(labels_cluster_5, class_names, 5)
percentagecalculator(labels_cluster_6, class_names, 6)
percentagecalculator(labels_cluster_7, class_names, 7)
percentagecalculator(labels_cluster_8, class_names, 8)
percentagecalculator(labels_cluster_9, class_names, 9)

Cluster 0 contains 19.09 %  4
Cluster 1 contains 17.09 %  7
Cluster 2 contains 16.32 %  1
Cluster 3 contains 19.83 %  6
Cluster 4 contains 28.11 %  0
Cluster 5 contains 30.26 %  8
Cluster 6 contains 30.57 %  9
Cluster 7 contains 19.14 %  7
Cluster 8 contains 14.94 %  2
Cluster 9 contains 22.0 %  5


In [0]:
["0airplane", "1automobile", "2bird", "3cat", "4deer", "5dog", "6frog", "7horse", "8ship", "9truck"]

['0airplane',
 '1automobile',
 '2bird',
 '3cat',
 '4deer',
 '5dog',
 '6frog',
 '7horse',
 '8ship',
 '9truck']

**K-means for 10 iterations**
* Cluster 0 contains 33.53 %  8
* Cluster 1 contains 33.27 %  0
* Cluster 2 contains 28.5 %  9
* Cluster 3 contains 15.27 %  2
* Cluster 4 contains 17.51 %  7
* Cluster 5 contains 18.79 %  4
* Cluster 6 contains 14.2 %  5
* Cluster 7 contains 19.01 %  6
* Cluster 8 contains 17.25 %  5
* Cluster 9 contains 17.37 %  1

**bold text**

**K-means for 100 iterations**

* Cluster 0 contains 32.97 %  8
* Cluster 1 contains 32.46 %  0
* Cluster 2 contains 30.45 %  9
* Cluster 3 contains 15.34 %  2
* Cluster 4 contains 18.81 %  7
* Cluster 5 contains 19.61 %  4
* Cluster 6 contains 15.34 %  5
* Cluster 7 contains 19.85 %  6
* Cluster 8 contains 19.54 %  5
* Cluster 9 contains 14.77 %  6

**K-means for 1000 iterations**

* Cluster 0 contains 32.97 %  8
* Cluster 1 contains 32.46 %  0
* Cluster 2 contains 30.45 %  9
* Cluster 3 contains 15.34 %  2
* Cluster 4 contains 18.81 %  7
* Cluster 5 contains 19.61 %  4
* Cluster 6 contains 15.34 %  5
* Cluster 7 contains 19.85 %  6
* Cluster 8 contains 19.54 %  5
* Cluster 9 contains 14.77 %  6

**Does k-means work similarly good?**

